# Building powerful image classification models using very little data

In this tutorial, we will present a few simple yet effective methods that you can use to build a powerful image classifier, using only very few training examples --just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:

training a small network from scratch (as a baseline)
using the bottleneck features of a pre-trained network
fine-tuning the top layers of a pre-trained network
This will lead us to cover the following Keras features:

fit_generator for training Keras a model using Python data generators
ImageDataGenerator for real-time data augmentation
layer freezing and model fine-tuning
...and more.

Our setup: only 2000 training examples (1000 per class)
We will start from the following setup:

a machine with Keras, SciPy, PIL installed. If you have a NVIDIA GPU that you can use (and cuDNN installed), that's great, but since we are working with few images that isn't strictly necessary.
a training data directory and validation data directory containing one subdirectory per image class, filled with .png or .jpg images:

To acquire a few hundreds or thousands of training images belonging to the classes you are interested in, one possibility would be to use the Flickr API to download pictures matching a given tag, under a friendly license.

In our examples we will use two sets of pictures, which we got from Kaggle: 1000 cats and 1000 dogs (although the original dataset had 12,500 cats and 12,500 dogs, we just took the first 1000 images for each class). We also use 400 additional samples from each class as validation data, to evaluate our models.

That is very few examples to learn from, for a classification problem that is far from simple. So this is a challenging machine learning problem, but it is also a realistic one: in a lot of real-world use cases, even small-scale data collection can be extremely expensive or sometimes near-impossible (e.g. in medical imaging). Being able to make the most out of very little data is a key skill of a competent data scientist.

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [4]:
!unzip -a dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
   creating: dogs-vs-cats/
   creating: dogs-vs-cats/preview/
   creating: dogs-vs-cats/train/
   creating: dogs-vs-cats/train/cats/
  inflating: dogs-vs-cats/train/cats/cat.9910.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.9720.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.12440.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.12141.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.12393.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.8463.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.4389.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.64.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.7101.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.459.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.7463.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1260.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6960.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1708.jpg  [binary]


  inflating: dogs-vs-cats/train/cats/cat.10263.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.5188.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.2543.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.552.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.2456.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.8811.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6924.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.9318.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.10302.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.3887.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.10318.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.11051.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1448.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.10523.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.10725.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.11985.jpg  [binary]
  inflating: dogs-vs-cats/train/ca

  inflating: dogs-vs-cats/train/cats/cat.4022.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.12031.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.3124.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.2688.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.11389.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.8174.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.7695.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.5363.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.4593.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.7040.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.7923.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.990.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.8105.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.850.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6884.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6013.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat

  inflating: dogs-vs-cats/train/cats/cat.6725.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.8215.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1564.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.7017.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6237.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.3255.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.2277.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1323.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.10799.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6674.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.5602.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.5021.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.9778.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.9762.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.12465.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.6852.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/c

  inflating: dogs-vs-cats/train/cats/cat.1788.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1789.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1790.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1791.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1792.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1793.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1794.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1795.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1796.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1797.jpg  [binary]
  inflating: dogs-vs-cats/train/cats/cat.1798.jpg  [binary]
   creating: dogs-vs-cats/train/dogs/
  inflating: dogs-vs-cats/train/dogs/dog.12472.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.12474.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.12475.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.12476.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.12477.jpg  [binar

...

  inflating: dogs-vs-cats/train/dogs/dog.10297.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.6308.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.11454.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.12485.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.6863.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.1192.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.1752.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.10023.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.7918.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.4722.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.2448.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.5494.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.9720.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.3069.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.599.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.9002.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/

  inflating: dogs-vs-cats/train/dogs/dog.12293.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.2843.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.7230.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.2440.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.2661.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.920.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.7265.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.3268.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.3736.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.6060.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.936.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.3061.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.7798.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.7810.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.5201.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog.11268.jpg  [binary]
  inflating: dogs-vs-cats/train/dogs/dog

  inflating: dogs-vs-cats/test/cats/cat.8593.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.746.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.9607.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.6485.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.7943.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.8326.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.6055.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.5509.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.4880.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.10951.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.4722.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.418.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.10240.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.6159.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.11490.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.6203.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.4726.jpg  [bina

  inflating: dogs-vs-cats/test/cats/cat.3350.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.2196.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.2339.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.1976.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.2802.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.4026.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.1427.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.6198.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.2841.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.3453.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.7999.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.6771.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.11801.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.2345.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.2009.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.11301.jpg  [binary]
  inflating: dogs-vs-cats/test/cats/cat.10635.jpg  [bi

  inflating: dogs-vs-cats/test/dogs/dog.10714.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.10480.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.8898.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.11270.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.7918.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.9758.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.5689.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.9212.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.7442.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.7450.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.8670.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.11512.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.5960.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.6515.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.12343.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.5508.jpg  [binary]
  inflating: dogs-vs-cats/test/dogs/dog.11800.jpg  

In [3]:
ls dogs-vs-cats/train

cats/ dogs/


In [4]:
img = load_img('dogs-vs-cats/train/dogs/dog.10016.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='dogs-vs-cats/preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [5]:
import tensorflow as tf

In [6]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K

In [7]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'dogs-vs-cats/train'
validation_data_dir = 'dogs-vs-cats/test'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [24]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
model.save_weights('first_try.h5')

Epoch 1/50
125/125 [==============================] - 41s 328ms/step - loss: 0.4365 - accuracy: 0.8075 - val_loss: 0.6209 - val_accuracy: 0.7812
Epoch 2/50
125/125 [==============================] - 50s 396ms/step - loss: 0.4278 - accuracy: 0.8165 - val_loss: 0.6642 - val_accuracy: 0.7788
Epoch 3/50
125/125 [==============================] - 51s 411ms/step - loss: 0.4363 - accuracy: 0.8085 - val_loss: 0.6533 - val_accuracy: 0.7538
Epoch 4/50
125/125 [==============================] - 57s 457ms/step - loss: 0.4296 - accuracy: 0.8220 - val_loss: 0.8203 - val_accuracy: 0.7700
Epoch 5/50
125/125 [==============================] - 52s 419ms/step - loss: 0.4299 - accuracy: 0.8135 - val_loss: 0.6507 - val_accuracy: 0.8000
Epoch 6/50
125/125 [==============================] - 48s 387ms/step - loss: 0.4349 - accuracy: 0.8150 - val_loss: 0.6784 - val_accuracy: 0.7450
Epoch 7/50
125/125 [==============================] - 49s 389ms/step - loss: 0.4303 - accuracy: 0.8155 - val_loss: 0.6698 - val_ac

# Part 2

In [32]:
from tensorflow.keras import applications
import numpy as np
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'dogs-vs-cats/train'
validation_data_dir = 'dogs-vs-cats/test'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [33]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [34]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    return model

In [35]:
save_bottlebeck_features()

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [36]:
model = train_top_model()

Epoch 1/50
125/125 [==============================] - 1s 10ms/step - loss: 0.6809 - accuracy: 0.7540 - val_loss: 0.2915 - val_accuracy: 0.8750
Epoch 2/50
125/125 [==============================] - 1s 9ms/step - loss: 0.4169 - accuracy: 0.8240 - val_loss: 0.2932 - val_accuracy: 0.8763
Epoch 3/50
125/125 [==============================] - 1s 9ms/step - loss: 0.3337 - accuracy: 0.8650 - val_loss: 0.2874 - val_accuracy: 0.8875
Epoch 4/50
125/125 [==============================] - 1s 9ms/step - loss: 0.2581 - accuracy: 0.8935 - val_loss: 0.2872 - val_accuracy: 0.8925
Epoch 5/50
125/125 [==============================] - 1s 9ms/step - loss: 0.2365 - accuracy: 0.9085 - val_loss: 0.3656 - val_accuracy: 0.8687
Epoch 6/50
125/125 [==============================] - 1s 9ms/step - loss: 0.2029 - accuracy: 0.9175 - val_loss: 0.3339 - val_accuracy: 0.8900
Epoch 7/50
125/125 [==============================] - 1s 9ms/step - loss: 0.1689 - accuracy: 0.9375 - val_loss: 0.3047 - val_accuracy: 0.9100
Epoch

In [38]:
pwd

'/Users/makhanov/Yandex.Disk.localized/week4/day5'

# Part 3

In [27]:
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
#import numpy as np
# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'cats_and_dogs_small/train'
validation_data_dir = 'cats_and_dogs_small/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
#model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
print('Model loaded.')


Model loaded.


In [25]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [29]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
#model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [30]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)      

In [34]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=2)

In [35]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[es])
model.save_weights('fine_tuned_cats_n_dogs.h5')

Epoch 1/50
125/125 [==============================] - 265s 2s/step - loss: 0.9245 - accuracy: 0.9065 - val_loss: 1.0857 - val_accuracy: 0.9137
Epoch 2/50
125/125 [==============================] - 576s 5s/step - loss: 0.7673 - accuracy: 0.9165 - val_loss: 1.0857 - val_accuracy: 0.9137
Epoch 3/50
125/125 [==============================] - 671s 5s/step - loss: 0.8195 - accuracy: 0.9220 - val_loss: 1.0857 - val_accuracy: 0.9137
